#  Install Dependencies and Bring in Data

In [1]:
#os module is a part of the standard library or stdlib-->it comes with your Python installation
import os                           
# tells Python to bring the pandas data analysis library into your current environment--> used for importing,managing,manipulating,analyzing datasets
import pandas as pd   
# It helps you implement best practices for data automation, model tracking, performance monitoring, and model retraining.
import tensorflow as tf
#to perform a wide variety of mathematical operations on numpy arrays
import numpy as np
#Warning messages are typically issued in situations where it is useful to alert the user of some condition in a program
import warnings
warnings.filterwarnings('ignore')    

In [2]:
#read_csv -->used to load a CSV file as a pandas dataframe
df = pd.read_csv(os.path.join('toxic-comment-classification-challenge','train.csv', 'train.csv'))

In [3]:
# to get the first 5 rows.
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Preprocess

In [4]:
from tensorflow.keras.layers import TextVectorization

In [5]:
X = df['comment_text']
y = df[df.columns[2:]].values

In [6]:
MAX_FEATURES = 200000 # number of words in the vocab

In [7]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

In [8]:
vectorizer.adapt(X.values)

In [9]:
vectorized_text = vectorizer(X.values)

In [10]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks

In [11]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

# Create Sequential Model

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [13]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))

In [14]:
model.compile(loss='BinaryCrossentropy', optimizer='Adam')

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [16]:
history = model.fit(train, epochs=1, validation_data=val)

In [17]:
from matplotlib import pyplot as plt

In [18]:
plt.figure(figsize=(8,5))
pd.DataFrame(history.history).plot()
plt.show()

# Make Predictions

In [19]:
input_text = vectorizer('I am going to hit you.')

In [20]:
res = model.predict(np.array([input_text]))

1/1 [==============================] - 1s 1s/step


In [21]:
(res > 0.5).astype(int)

array([[1, 0, 1, 1, 0, 0]])

In [22]:
batch_X, batch_y = test.as_numpy_iterator().next()

In [23]:
(model.predict(batch_X) > 0.5).astype(int)

1/1 [==============================] - 0s 211ms/step


array([[1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 0, 0],
       [1, 0, 1, 1, 1, 0],
       [1, 0, 1, 1, 0, 0]])

In [24]:
res.shape

(1, 6)

# Evaluate Model

In [25]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [26]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [27]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    X_true, y_true = batch
    # Make a prediction 
    yhat = model.predict(X_true)
    
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 182ms/step


1/1 [==============================] - 0s 168ms/step


1/1 [==============================] - 0s 159ms/step


1/1 [==============================] - 0s 204ms/step


1/1 [==============================] - 0s 156ms/step


1/1 [==============================] - 0s 151ms/step


1/1 [==============================] - 0s 160ms/step


In [28]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.05006758123636246, Recall:0.7220339179039001, Accuracy:0.0


# Test and Gradio

In [29]:
import tensorflow as tf
import gradio as gr

In [30]:
model.save('toxicity.h5')

In [31]:
model = tf.keras.models.load_model('toxicity.h5')

In [32]:
input_str = vectorizer('hey i freaken hate you!')

In [33]:
res = model.predict(np.expand_dims(input_str,0))

1/1 [==============================] - 1s 959ms/step


In [34]:
res

array([[0.5014692 , 0.49931467, 0.5014247 , 0.50181305, 0.5003121 ,
        0.49804255]], dtype=float32)

In [35]:
def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text

In [36]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                        outputs='text')

In [37]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://301ae2692f85a32691.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 128ms/step
